In [8]:
import pymorphy2
import pandas as pd
from tqdm import tqdm
import string
import numpy as np
import faiss
import os
import fasttext
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

In [2]:
dataset = pd.read_csv('../dataset_2.csv')

In [3]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
tqdm.pandas()

class Data(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = np.array(data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]
        return text

class Searcher():
    def __init__(self, data, search_type, b=0.75, k=2, texts=None, inv_index=None):
        self.morph = pymorphy2.MorphAnalyzer()
        self.punctuation = string.punctuation + '…' + '«' + '»' + '—'
        self.stop_words = stopwords.words('russian')
        self.use_lemmatization = search_type == 'bm25_lemm'
        self.search_type = search_type
        
        if search_type == 'vec_fasttext':
            self.vectorizer = fasttext.load_model("../cc.ru.300.bin")
        elif search_type == 'vec_bert':
            self.tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
            self.vectorizer = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

        self.k = k
        self.b = b

        self.data = data # исходные данные

        if texts is None:
            print('Preprocessing texts...')
            if search_type == 'vec_bert' or search_type == 'vec_fasttext':
                self.texts = data.text.progress_apply(self.preprocess_text)
            else:
                self.texts = data.text.progress_apply(self.preprocess_text) # очищенные тексты в нормальной форме
                
        else:
            self.texts = texts

        if inv_index is None:
            print('Counting inverted index...')
            
            if search_type == 'vec_fasttext':
                texts_vectorized = np.zeros([len(self.texts), self.vectorizer.get_dimension()])
                for doc_id, doc in enumerate(tqdm(self.texts)):
                    
                    texts_vectorized[doc_id] = self.vectorizer[str(doc)]

                inv_index = faiss.IndexFlatL2(texts_vectorized.shape[1])
                inv_index.add(texts_vectorized)

                faiss.write_index(inv_index, "../vec.index")
                
            elif search_type == 'vec_bert':
                
                embeddings = self.vectorize(self.texts)

            elif search_type == 'bm25_lemm':
                inv_index = {}

                for doc_id, doc in enumerate(tqdm(self.texts)):
                    tokens = doc.split()
                    for token in tokens:
                        if token in inv_index:
                            inv_index[token].add(doc_id)
                        else:
                            inv_index[token] = set([doc_id])
            else:
                raise ValueError()
        
        self.inv_index = inv_index
    
    def vectorize(self, df):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"{device} is used")
        self.vectorizer.to(device)
        self.vectorizer.eval()
        ldr_data = Data(data=df)
        loader = DataLoader(dataset=ldr_data, shuffle=False, batch_size=min([10, len(df)]))
        embeddings = []
        with torch.no_grad():
            for i, inputs in tqdm(enumerate(loader)):
                batch = self.tokenizer(list(inputs), truncation=True, padding=True, return_tensors='pt', max_length=24)
                input_ids = batch['input_ids']
                input_ids = input_ids.to(device, dtype=torch.long)
                batch['input_ids'] = input_ids

                token_type_ids = batch['token_type_ids']
                token_type_ids = token_type_ids.to(device, dtype=torch.long)
                batch['token_type_ids'] = token_type_ids

                attention_mask = batch['attention_mask']
                attention_mask = attention_mask.to(device, dtype=torch.long)
                batch['attention_mask'] = attention_mask

                preds = self.vectorizer(**batch)
                preds = mean_pooling(preds, attention_mask)
                outputs = preds.cpu().detach().numpy()
                embeddings.extend(outputs)
        
        return embeddings

    def preprocess_text(self, text):
        tokens = str(text).lower().translate(str.maketrans('', '', self.punctuation)).split()
        res = ''

        if self.use_lemmatization:
            for token in tokens:
                p = self.morph.parse(token)[0].normal_form
                if p not in self.stop_words:
                    res += ' ' + p
        else:
            for token in tokens:
                if token not in self.stop_words:
                    res += ' ' + token
        
        return res


    def idf(self, terms, documents, eps=-0.01):
        idfs = np.asarray([len(documents) / (len(self.inv_index[term]) + eps) for term in terms])
        return np.log(idfs)


    def tf(self, terms, doc_ids):
        # можно посплитить текст в init и не сплитить каждый раз
        def count(text):
            freqs = dict.fromkeys(terms, 0)
            text = text.split() 
            for term in text:
                if term in freqs.keys():
                    freqs[term] += 1
            
            return np.asarray([term_freq[1] for term_freq in freqs.items()]) / len(text)

        docs = self.texts[doc_ids]

        return np.stack(docs.apply(count).values)
    
    def tf_idf(self, terms, documents):
        tf_scores = self.tf(terms, documents)
        idf_scores = self.idf(terms, documents)

        scores = tf_scores * idf_scores
        return scores

    def bm25(self, terms, documents):
        
        tf_scores = self.tf(terms, documents)
        idf_scores = self.idf(terms, documents)
        lens = self.texts[documents].apply(lambda x: len(x.split())).to_numpy()

        scores = np.zeros(lens.shape)
        for i, term in enumerate(terms):
            a = (tf_scores[:, i] * (self.k + 1))
            b = (tf_scores[:, i] + self.k * (1 - self.b + self.b * lens / lens.mean()))
            c = idf_scores[i]
            scores += a / b * c

        return scores

    def search(self, query):
        if self.search_type != 'vec_fasttext':
            terms = self.preprocess_text(query).split()

            global_ids = set()
            for term in terms:
                global_ids.update(self.inv_index[term])

            global_ids = np.asarray(list(global_ids))
            scores = self.bm25(terms, global_ids)#.sum(axis=1)

            local_ids = np.argsort(scores, axis=0).squeeze() # сортируем документы по возрастанию tf-idf
            

            return self.data.iloc[global_ids[local_ids][-10:]]
        
        else:
            query_vector = self.vectorizer[query]

            distances, indicies = self.inv_index.search(np.expand_dims(query_vector, 0), 10)

            print(distances)
            print(indicies[0])

            return self.data.iloc[indicies[0]]
        
        

In [22]:
s = Searcher(dataset, search_type='vec_fasttext')

Preprocessing texts...


100%|████████████████████████████████████████████████████████████████████████████| 10385/10385 [00:35<00:00, 295.04it/s]


Counting inverted index...


100%|████████████████████████████████████████████████████████████████████████████| 10385/10385 [00:37<00:00, 274.26it/s]


In [ ]:
texts = s.texts

In [29]:
s.search('Зорот (Xoroth) — мир из вселенной Warcraft. Эта планета принадлежит Пылающему Легиону, в частности, натрезимам, и является родиной коней погибели, которых чернокнижники используют в качестве верховых лошадей.')

[[0.00336832 0.00585805 0.00588941 0.00591027 0.00594001 0.00594629
  0.00595441 0.00595623 0.00596606 0.00597592]]
[6976 9864 1288 8664 9850 6135 3309 5728 1092 5097]


,Unnamed: 0,title,link,text
6976,6976,Зорот,/wiki/%D0%97%D0%BE%D1%80%D0%BE%D1%82,Зорот (Xoroth) — мир из вселенной Warcraft. Эт...
9864,9864,Атон,/wiki/%D0%90%D1%82%D0%BE%D0%BD,Атон — египетский бог в форме солнечного диска...
1288,1288,Астрал,/wiki/%D0%90%D1%81%D1%82%D1%80%D0%B0%D0%BB,"Астрал (Тонкий мир, Незримый мир, и т. д., пет..."
8664,8664,Чужое небо,/wiki/%D0%A7%D1%83%D0%B6%D0%BE%D0%B5_%D0%BD%D0...,"В небе сиял Юпитер — стояло полнолуние, вернее..."
9850,9850,Иштар,/wiki/%D0%98%D1%88%D1%82%D0%B0%D1%80,"Иштар, также известная как Инанна, Астарта (ра..."
6135,6135,Верховный бог,/wiki/%D0%92%D0%B5%D1%80%D1%85%D0%BE%D0%B2%D0%...,"Слушайте слово мое, и боги небес, и богини: Я..."
3309,3309,Оман,/wiki/%D0%9E%D0%BC%D0%B0%D0%BD,Оман — унылая страна кагбе в регионе Персидско...
5728,5728,Коробочка фотонов,/wiki/%D0%9A%D0%BE%D1%80%D0%BE%D0%B1%D0%BE%D1%...,Коробочка фотонов (реже мешок или весовые част...
1092,1092,Венера,/wiki/%D0%92%D0%B5%D0%BD%D0%B5%D1%80%D0%B0,Следующая по влиянию на умы фантастов планета ...
5097,5097,Недотанк,/wiki/%D0%9D%D0%B5%D0%B4%D0%BE%D1%82%D0%B0%D0%...,Основные составляющие танка — умение держать у...


In [23]:
s.search('Зорот (Xoroth) — мир из вселенной Warcrafxt. Эта планета принадлежит Пылающему Легиону, в частности, натрезимам, и является родиной коней погибели, которых чернокнижники используют в качестве верховых лошадей.')

[[0.00336832 0.00585805 0.00588941 0.00591027 0.00594001 0.00594629
  0.00595441 0.00595623 0.00596606 0.00597592]]
[6976 9864 1288 8664 9850 6135 3309 5728 1092 5097]


,Unnamed: 0,title,link,text
6976,6976,Зорот,/wiki/%D0%97%D0%BE%D1%80%D0%BE%D1%82,Зорот (Xoroth) — мир из вселенной Warcraft. Эт...
9864,9864,Атон,/wiki/%D0%90%D1%82%D0%BE%D0%BD,Атон — египетский бог в форме солнечного диска...
1288,1288,Астрал,/wiki/%D0%90%D1%81%D1%82%D1%80%D0%B0%D0%BB,"Астрал (Тонкий мир, Незримый мир, и т. д., пет..."
8664,8664,Чужое небо,/wiki/%D0%A7%D1%83%D0%B6%D0%BE%D0%B5_%D0%BD%D0...,"В небе сиял Юпитер — стояло полнолуние, вернее..."
9850,9850,Иштар,/wiki/%D0%98%D1%88%D1%82%D0%B0%D1%80,"Иштар, также известная как Инанна, Астарта (ра..."
6135,6135,Верховный бог,/wiki/%D0%92%D0%B5%D1%80%D1%85%D0%BE%D0%B2%D0%...,"Слушайте слово мое, и боги небес, и богини: Я..."
3309,3309,Оман,/wiki/%D0%9E%D0%BC%D0%B0%D0%BD,Оман — унылая страна кагбе в регионе Персидско...
5728,5728,Коробочка фотонов,/wiki/%D0%9A%D0%BE%D1%80%D0%BE%D0%B1%D0%BE%D1%...,Коробочка фотонов (реже мешок или весовые част...
1092,1092,Венера,/wiki/%D0%92%D0%B5%D0%BD%D0%B5%D1%80%D0%B0,Следующая по влиянию на умы фантастов планета ...
5097,5097,Недотанк,/wiki/%D0%9D%D0%B5%D0%B4%D0%BE%D1%82%D0%B0%D0%...,Основные составляющие танка — умение держать у...


In [30]:
s.search('война судного дня')

[[0.100796   0.10161673 0.10191935 0.10303252 0.10348049 0.10446803
  0.10454261 0.10461313 0.10477074 0.10478412]]
[ 4910  1328  7344  8077  1861 10362  5706  6196  9515  3815]


,Unnamed: 0,title,link,text
4910,4910,Циник,/wiki/%D0%A6%D0%B8%D0%BD%D0%B8%D0%BA,"Циники (Циничные люди) — приверженцы цинизма, ..."
1328,1328,Сотона,/wiki/%D0%A1%D0%BE%D1%82%D0%BE%D0%BD%D0%B0,"Рыба гниёт с головы, но чистят её с хвоста. С..."
7344,7344,MrGoodmobile,/wiki/MrGoodmobile,"MrGoodmobile — укропский видеоблогер, снимающи..."
8077,8077,Смегма,/wiki/%D0%A1%D0%BC%D0%B5%D0%B3%D0%BC%D0%B0,"Смегма (англ. Dick Cheese, более известна как ..."
1861,1861,Копипаста:Lurkmore:Contacts,/wiki/%D0%9A%D0%BE%D0%BF%D0%B8%D0%BF%D0%B0%D1%...,Официальным пресс-секретарем Луркоморья являет...
10362,10362,Евгений Викторович Кошевой,/wiki/%D0%95%D0%B2%D0%B3%D0%B5%D0%BD%D0%B8%D0%...,Евгений Викторович Кошевой - лысый придурок и ...
5706,5706,Тряпка-кун,/wiki/%D0%A2%D1%80%D1%8F%D0%BF%D0%BA%D0%B0-%D0...,Тряпка-кун — одна из самых громких деанонимиза...
6196,6196,Нирвана,/wiki/%D0%9D%D0%B8%D1%80%D0%B2%D0%B0%D0%BD%D0%B0,"Нирвана — строго говоря, раем не является. В Б..."
9515,9515,Надписи на снарядах,/wiki/%D0%9D%D0%B0%D0%B4%D0%BF%D0%B8%D1%81%D0%...,"Надписи на снарядах — явление, при котором на ..."
3815,3815,Шулхан Арух,/wiki/%D0%A8%D1%83%D0%BB%D1%85%D0%B0%D0%BD_%D0...,Я задался вопросом: на каком основании евреи б...


In [27]:
dataset.iloc[7840].text

'Это страница неоднозначности. Ниже можно прочитать список значений и выбрать из них интересующее. Бывает:  смешной по-настоящему страшный'

In [31]:
s.search('украинский кризис')

[[0.06356399 0.06866173 0.07268341 0.07331744 0.0736462  0.07372062
  0.07374732 0.07375086 0.07394659 0.07399531]]
[ 3650  9429  9609 10333  9268  1755  2436 10357   649  4280]


,Unnamed: 0,title,link,text
3650,3650,Кризис,/wiki/%D0%9A%D1%80%D0%B8%D0%B7%D0%B8%D1%81,"Большой пиздец не за горами, и волны кризисов ..."
9429,9429,Украинский кризис,/wiki/%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D1%...,Украинский кризис — политический кризис с учас...
9609,9609,Запрет айфонов в России,/wiki/%D0%97%D0%B0%D0%BF%D1%80%D0%B5%D1%82_%D0...,Запрет айфонов в России — новая интересная ини...
10333,10333,Новороссия,/wiki/%D0%9D%D0%BE%D0%B2%D0%BE%D1%80%D0%BE%D1%...,Новороссия — название для проекта независимого...
9268,9268,Я случайно X,/wiki/%D0%AF_%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0...,Я случайно X (англ. I accidentally X) — мем ан...
1755,1755,Шаблон:Спасено с Лурка/More,/wiki/%D0%A8%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD:%D0...,"""Вдруг война, а я уставший..."" .onion 1 мая 1...."
2436,2436,Крикун,/wiki/%D0%9A%D1%80%D0%B8%D0%BA%D1%83%D0%BD,"Крикун (Выпускник, Яростный Школьник, Кри-кун)..."
10357,10357,Игорь Мосийчук,/wiki/%D0%98%D0%B3%D0%BE%D1%80%D1%8C_%D0%9C%D0...,"Украинские клоуны уже весь мир смешат, то с па..."
649,649,Почему титаны слишком большого размера? (Хрони...,/wiki/%D0%9F%D0%BE%D1%87%D0%B5%D0%BC%D1%83_%D1...,Почему титаны слишком большого размера? — расп...
4280,4280,Закрытие Сохнута в России,/wiki/%D0%97%D0%B0%D0%BA%D1%80%D1%8B%D1%82%D0%...,Закрытие Сохнута в России — закрытие еврейског...


In [33]:
s.search('война в украине')

[[0.15854478 0.15978475 0.16024865 0.16064066 0.1608152  0.16151662
  0.1615301  0.16248792 0.16379516 0.16472024]]
[ 1009  1008  8077  9429  1328 10362  5155  7344 10357 10342]


,Unnamed: 0,title,link,text
1009,1009,Алфавит Двача,/wiki/%D0%90%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%...,Эта галерея алфавита Двача. В уютненьком есть ...
1008,1008,Мост в Терабитию/Алфавит,/wiki/%D0%9C%D0%BE%D1%81%D1%82_%D0%B2_%D0%A2%D...,Эта подстатья об алфавите АнныСофии. В уютнень...
8077,8077,Смегма,/wiki/%D0%A1%D0%BC%D0%B5%D0%B3%D0%BC%D0%B0,"Смегма (англ. Dick Cheese, более известна как ..."
9429,9429,Украинский кризис,/wiki/%D0%A3%D0%BA%D1%80%D0%B0%D0%B8%D0%BD%D1%...,Украинский кризис — политический кризис с учас...
1328,1328,Сотона,/wiki/%D0%A1%D0%BE%D1%82%D0%BE%D0%BD%D0%B0,"Рыба гниёт с головы, но чистят её с хвоста. С..."
10362,10362,Евгений Викторович Кошевой,/wiki/%D0%95%D0%B2%D0%B3%D0%B5%D0%BD%D0%B8%D0%...,Евгений Викторович Кошевой - лысый придурок и ...
5155,5155,"Внутри меньше, чем снаружи",/wiki/%D0%92%D0%BD%D1%83%D1%82%D1%80%D0%B8_%D0...,В компьютерных играх нередко можно вломиться в...
7344,7344,MrGoodmobile,/wiki/MrGoodmobile,"MrGoodmobile — укропский видеоблогер, снимающи..."
10357,10357,Игорь Мосийчук,/wiki/%D0%98%D0%B3%D0%BE%D1%80%D1%8C_%D0%9C%D0...,"Украинские клоуны уже весь мир смешат, то с па..."
10342,10342,Закарпатье,/wiki/%D0%97%D0%B0%D0%BA%D0%B0%D1%80%D0%BF%D0%...,Закарпатье — самый западный и самый дальний от...


In [42]:
s.search('конституция российской федерации')

[[0.02322684 0.0241304  0.02451413 0.02452654 0.02470964 0.02480914
  0.02483547 0.02484737 0.02490055 0.02491833]]
[ 2203  3452  3234  5298  3201  7264  2825  1034  4507 10200]


,Unnamed: 0,title,link,text
2203,2203,Педерация,/wiki/%D0%9F%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%...,Педерация — распространённое искажение слова «...
3452,3452,Брайтон-Бич,/wiki/%D0%91%D1%80%D0%B0%D0%B9%D1%82%D0%BE%D0%...,Брайтон-Бич (Маленькая Одесса) — русско-украин...
3234,3234,Дружественные страны России,/wiki/%D0%94%D1%80%D1%83%D0%B6%D0%B5%D1%81%D1%...,"Дружественные страны России, — признанные офиц..."
5298,5298,Posmotre.li:Навигация,/wiki/Posmotre.li:%D0%9D%D0%B0%D0%B2%D0%B8%D0%...,Мета Основы Основы Жанры Сюжет Производное тво...
3201,3201,Привислинский край,/wiki/%D0%9F%D1%80%D0%B8%D0%B2%D0%B8%D1%81%D0%...,Привислинский край (также Привислянский край; ...
7264,7264,Иван Лизан,/wiki/%D0%98%D0%B2%D0%B0%D0%BD_%D0%9B%D0%B8%D0...,Иван Лизан (ivan_lizan) — российский экономист...
2825,2825,Герман Bealik,/wiki/%D0%93%D0%B5%D1%80%D0%BC%D0%B0%D0%BD_Bealik,"Герман Bealik — пользователь Ответы@mail.ru, н..."
1034,1034,Старая копипаста:Ичуньская Компания,/wiki/%D0%A1%D1%82%D0%B0%D1%80%D0%B0%D1%8F_%D0...,250-я Ичуньская Приграничная Синтетическая Тор...
4507,4507,LMAO,/wiki/LMAO,LMAO — аббревиатура от англоязычной фразы «lau...
10200,10200,Сборная России по футболу,/wiki/%D0%A1%D0%B1%D0%BE%D1%80%D0%BD%D0%B0%D1%...,Сборная России по футболу — собирательное назв...


In [34]:
s.search('видеоигры')

[[0.37629652 0.3780882  0.37857202 0.37966228 0.37989593 0.3799671
  0.37997022 0.38007995 0.38013956 0.38020948]]
[5038 7780 6796  891 6971 6772 6643 8914 5037 7155]


,Unnamed: 0,title,link,text
5038,5038,Симулятор,/wiki/%D0%A1%D0%B8%D0%BC%D1%83%D0%BB%D1%8F%D1%...,"Симулятор — жанр компьютерных игр, суть которы..."
7780,7780,Фанарт,/wiki/%D0%A4%D0%B0%D0%BD%D0%B0%D1%80%D1%82,"Фанарт (fan art, англ. «фанатское искусство») ..."
6796,6796,Киновселенная Марвел,/wiki/%D0%9A%D0%B8%D0%BD%D0%BE%D0%B2%D1%81%D0%...,Кинематографическая вселенная Марвел (Marvel C...
891,891,"Ой-вей, нiнгенъ сакеръ","/wiki/%D0%9E%D0%B9-%D0%B2%D0%B5%D0%B9,_%D0%BDi...","Ой-вей, нiнгенъ сакеръ (moon. へいへい人間賛歌) — мем ..."
6971,6971,Расы вселенной Warcraft,/wiki/%D0%A0%D0%B0%D1%81%D1%8B_%D0%B2%D1%81%D0...,Люди Дворфы Ночные эльфы Гномы Дренеи Воргены ...
6772,6772,Метрополис,/wiki/%D0%9C%D0%B5%D1%82%D1%80%D0%BE%D0%BF%D0%...,Это страница неоднозначности. Ниже можно прочи...
6643,6643,Sid Meier's Starships,/wiki/Sid_Meier%27s_Starships,Sid Meier’s Starships — сиквел к Sid Meier’s C...
8914,8914,Джейсон Стейтем,/wiki/%D0%94%D0%B6%D0%B5%D0%B9%D1%81%D0%BE%D0%...,Джейсон Стейтем (англ. Jason Statham; род. 26 ...
5037,5037,Метроидвания,/wiki/%D0%9C%D0%B5%D1%82%D1%80%D0%BE%D0%B8%D0%...,Метроидвания — поджанр платформеров с геймплее...
7155,7155,Garfield,/wiki/Garfield,«Гарфилд» (англ. Garfield) — это газетный коми...


In [50]:
s.search('все прогрессивное человечество')

[[0.02552456 0.02793368 0.0279752  0.02817386 0.02845105 0.02851618
  0.02867845 0.02883737 0.0288555  0.02889591]]
[7646 2254 9776 3182 5842 4741 4345 9775 5622 4682]


,Unnamed: 0,title,link,text
7646,7646,Всё прогрессивное человечество,/wiki/%D0%92%D1%81%D1%91_%D0%BF%D1%80%D0%BE%D0...,Всё прогрессивное человечество — советский мем...
2254,2254,Сколько человек может прожить без сна,/wiki/%D0%A1%D0%BA%D0%BE%D0%BB%D1%8C%D0%BA%D0%...,Сколько человек может прожить без сна — распро...
9776,9776,Воскрешение,/wiki/%D0%92%D0%BE%D1%81%D0%BA%D1%80%D0%B5%D1%...,Воскрешение — возвращение человека в жизни пос...
3182,3182,Англосаксонские владения,/wiki/%D0%90%D0%BD%D0%B3%D0%BB%D0%BE%D1%81%D0%...,"Англосаксонские владения — союз государств, ко..."
5842,5842,Дизайнеры жизни,/wiki/%D0%94%D0%B8%D0%B7%D0%B0%D0%B9%D0%BD%D0%...,Дизайнеры жизни (Life Planners) — конспирологи...
4741,4741,Homo overclocked,/wiki/Homo_overclocked,Homo overclocked (человек разогнанный) — харак...
4345,4345,Еврейский музей и центр толерантности,/wiki/%D0%95%D0%B2%D1%80%D0%B5%D0%B9%D1%81%D0%...,Еврейский музей и центр толерантности — москов...
9775,9775,Владыка,/wiki/%D0%92%D0%BB%D0%B0%D0%B4%D1%8B%D0%BA%D0%B0,"Владыка — мощное слово, обозначающее действите..."
5622,5622,Социальный рейтинг,/wiki/%D0%A1%D0%BE%D1%86%D0%B8%D0%B0%D0%BB%D1%...,Социальный рейтинг (Социальный кредит) — важны...
4682,4682,Нейтралы,/wiki/%D0%9D%D0%B5%D0%B9%D1%82%D1%80%D0%B0%D0%...,"Нейтралы — люди, придерживающиеся политики ней..."


In [37]:
s.search('автомобили')

[[0.24243669 0.24249664 0.24253696 0.24265097 0.24265297 0.24265862
  0.24276732 0.24287319 0.243154   0.24320997]]
[ 5298 10377  9765  3602 10305  5251  3464  9225  2116  9588]


,Unnamed: 0,title,link,text
5298,5298,Posmotre.li:Навигация,/wiki/Posmotre.li:%D0%9D%D0%B0%D0%B2%D0%B8%D0%...,Мета Основы Основы Жанры Сюжет Производное тво...
10377,10377,Daewoo Lanos,/wiki/Daewoo_Lanos,"Daewoo Lanos (редк. Doninvest Assol, хохл. Sen..."
9765,9765,Четыре всадника Апокалипсиса,/wiki/%D0%A7%D0%B5%D1%82%D1%8B%D1%80%D0%B5_%D0...,Четыре всадника Апокалипсиса — концепт четырёх...
3602,3602,Кинжал (гиперзвуковая ракета),/wiki/%D0%9A%D0%B8%D0%BD%D0%B6%D0%B0%D0%BB_(%D...,"Кинжал(или 9-А-7660, Х-47М2) — сверхмощная кул..."
10305,10305,ТАЗ,/wiki/%D0%A2%D0%90%D0%97,Тазы (ВАЗ-Лада)- адские создания от не менее а...
5251,5251,Ракета летит медленно,/wiki/%D0%A0%D0%B0%D0%BA%D0%B5%D1%82%D0%B0_%D0...,Чак Норрис может поймать летящую в него ракету...
3464,3464,Охотничьи угодья,/wiki/%D0%9E%D1%85%D0%BE%D1%82%D0%BD%D0%B8%D1%...,"Охотничьи угодья — та или иная территория, выд..."
9225,9225,New School (сингл),/wiki/New_School_(%D1%81%D0%B8%D0%BD%D0%B3%D0%BB),New School — сингл российских рэп-исполнителей...
2116,2116,Вышиватник,/wiki/%D0%92%D1%8B%D1%88%D0%B8%D0%B2%D0%B0%D1%...,"Вышиватник (укр. вишіватнік, от слов «вышиванк..."
9588,9588,HTC,/wiki/HTC,HTC Corporation (бывшая High Tech Computer Cor...
